In [ ]:
import os
from itertools import compress

import numpy
import pandas
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.cluster import AgglomerativeClustering
from MulticoreTSNE import MulticoreTSNE as TSNE
from plotly import graph_objects
from plotly import offline as plotly_offline

from scrapi.dataset import Gene_Expression_Dataset as GED
from scrapi import plotting as scrap_plotting

from pepars.plotting import plotting
plotting.init_notebook_mode()

In [ ]:
# The sample to perform debris filtering on
SAMPLE = "AM1"

# Pick a cell number range. The first number should be a number higher than how many cells you expect.
# The second number should be as low as possible, such that you are confident that you will still get
# all of the cell types you're interested in
CELL_RANGE = [12000, 1000]

# How many sources of noise with unique transcript count profile you expect
# If you only expect empty droplets, this should be 1. If you expect empty droplets + debris, 2
NUM_SOURCES_OF_NOISE_EXPECTED = 3

# How many cell types you expect at the top level of your hierarchy.
NUM_CELL_TYPES_EXPECTED = 4

# How many clusters to try to separate into. Recommend at least num sources of noise, plus
# each major cell type, plus each subtype one level below major. For blood, we do
# noise (3) + cell types (4) + cell subtypes (2*cell types = 8) = 15
MAX_NUM_CLUSTERS = 15

In [ ]:
# Load the dataset
ged = GED(os.path.join("data", SAMPLE))

# Grab the transcript count matrix and total transcript count vector - we'll be using this a lot
transcript_counts = ged.get_cell_transcript_counts()
total_transcript_counts = ged.get_cell_total_transcript_counts()

min_num_clusters = NUM_SOURCES_OF_NOISE_EXPECTED + NUM_CELL_TYPES_EXPECTED

In [ ]:
# Look at the transcript vs cell count plot

scrap_plotting.plot_barcode_transcript_counts(
    total_transcript_counts,
    max_points=1000,
    elbow=False,
    interactive=True
)

In [ ]:
# Sort the transcript counts to get our transcript count thresholds based on our cell range estimate

sorted_transcript_counts = numpy.sort(total_transcript_counts)
transcript_count_thresholds = int(sorted_transcript_counts[-CELL_RANGE[0]]), int(sorted_transcript_counts[-CELL_RANGE[1]])
debris_transcript_count_threshold = transcript_count_thresholds[0]

In [ ]:
# Get a list of cells above the threshold
cell_indices_above_threshold = total_transcript_counts > debris_transcript_count_threshold

# Get the full gene count matrix and total transcript counts associated with these cells
filtered_cell_gene_counts = transcript_counts[cell_indices_above_threshold, :]
filtered_total_transcript_counts = total_transcript_counts[cell_indices_above_threshold]

print("%i cell(s) after threshold filtering" % filtered_cell_gene_counts.shape[0])

# Filter out zero genes to make it easier on PCA
non_zero_genes = filtered_cell_gene_counts.sum(axis=0) > 0
filtered_cell_gene_counts = filtered_cell_gene_counts[:, non_zero_genes]

print("%i gene(s) after zero filtering" % filtered_cell_gene_counts.shape[1])

# Normalize the cell gene counts - first divide by the sum to get transcripts per cell
filtered_cell_gene_counts.divide(filtered_cell_gene_counts.sum(axis=1))

# Multiply by a factor to separate single counts from zero counts
filtered_cell_gene_counts.multiply(5000)

# Add one before taking log
filtered_cell_gene_counts.add(1)

# Log scale
filtered_cell_gene_counts.log10()

print("Dimensionality reduction via PCA")

pca = PCA(n_components=50)
transformed_PCA = pca.fit_transform(filtered_cell_gene_counts.to_array())

print("Dimensionality reduction via tSNE")

transformed_tSNE = TSNE(
    verbose=True, perplexity=30, n_components=2, n_jobs=4).\
    fit_transform(transformed_PCA)

In [ ]:
# Attempt agglomerative clustering for a range of clusters and find the highest one

cluster_range = range(min_num_clusters, MAX_NUM_CLUSTERS)

silhouette_scores = []

for num_clusters in cluster_range:

    print("Testing %i clusters" % num_clusters)

    clusterer = AgglomerativeClustering(n_clusters=num_clusters)
    clusters = clusterer.fit_predict(transformed_PCA)

    silhouette_score = metrics.silhouette_score(transformed_PCA, clusters)
    silhouette_scores.append(silhouette_score)

num_clusters = cluster_range[numpy.argmax(silhouette_scores)]

plotting.plot_scatter(
    list(cluster_range),
    silhouette_scores,
    interactive=True,
    title="Silhouette distance for threshold %i" % debris_transcript_count_threshold
)

In [ ]:
# Take the highest number of clusters found and proceed

clusterer = AgglomerativeClustering(n_clusters=num_clusters)
clusters = clusterer.fit_predict(transformed_PCA)

x_values = []
y_values = []

for cluster in range(num_clusters):

    cluster_points = transformed_tSNE[clusters == cluster, :]

    x_values.append(cluster_points[:, 0])
    y_values.append(cluster_points[:, 1])

plotting.plot_scatter(
    x_values,
    y_values,
    interactive=True,
    title="Clustering for transcript count threshold %i" % debris_transcript_count_threshold,
    trace_names=list(range(num_clusters))
)

In [ ]:
# Plot the mitochondrial gene ratio (for QC)

mt_genes = []

for gene in ged.get_genes():
    if gene.lower().startswith("mt-"):
        mt_genes.append(gene)
        
mt_ratio = transcript_counts[cell_indices_above_threshold, mt_genes].sum(axis=1)/\
    transcript_counts[cell_indices_above_threshold, :].sum(axis=1)

figure = graph_objects.Figure(data=graph_objects.Scatter(
    x=transformed_tSNE[:, 0],
    y=transformed_tSNE[:, 1],
    mode="markers",
    text=mt_ratio,
    marker=dict(
        size=6,
        color=mt_ratio,
        colorscale="Viridis",
        showscale=True
    )
))

plotly_offline.iplot(figure)

In [ ]:
# Plot the HBB ratio (for QC)
        
HBB_ratios = transcript_counts[cell_indices_above_threshold, "HBB"].sum(axis=1)/\
    transcript_counts[cell_indices_above_threshold, :].sum(axis=1)

figure = graph_objects.Figure(data=graph_objects.Scatter(
    x=transformed_tSNE[:, 0],
    y=transformed_tSNE[:, 1],
    mode="markers",
    text=HBB_ratios,
    marker=dict(
        size=6,
        color=HBB_ratios,
        colorscale="Viridis",
        showscale=True
    )
))

plotly_offline.iplot(figure)

In [ ]:
# Plot the transcript counts (for QC)
        
transcript_count_sums = transcript_counts[cell_indices_above_threshold, :].sum(axis=1)

figure = graph_objects.Figure(data=graph_objects.Scatter(
    x=transformed_tSNE[:, 0],
    y=transformed_tSNE[:, 1],
    mode="markers",
    text=transcript_count_sums,
    marker=dict(
        size=6,
        color=transcript_count_sums,
        colorscale="Viridis",
        showscale=True
    )
))

plotly_offline.iplot(figure)

In [ ]:
# Get a range of thresholds to test from the min to the max - this is to establish a slope of cell
# dropoff per cluster as we increase the transcript count threshold
fine_grain_transcript_count_thresholds = range(debris_transcript_count_threshold, transcript_count_thresholds[1], 50)

# A dataframe that lists the number of cells in each cluster above the thresholds
cluster_size_from_most_to_least = pandas.DataFrame(
    index=list(range(num_clusters)),
    columns=list(fine_grain_transcript_count_thresholds)
)

for threshold in fine_grain_transcript_count_thresholds:

    for cluster in range(num_clusters):
        
        # Get how many cells are in this cluster at this threshold
        cluster_cell_counts = \
            filtered_total_transcript_counts[(clusters == cluster) & (filtered_total_transcript_counts > threshold)]

        cluster_size_from_most_to_least.loc[cluster, threshold] = cluster_cell_counts.shape[0]

In [ ]:
# Inspect to see the cell dropoff curve for the different clusters

plotting.plot_scatter(
    [cluster_size_from_most_to_least.columns]*num_clusters,
    cluster_size_from_most_to_least.values,
    interactive=True,
    trace_names=list(range(num_clusters)),
    x_axis_title="Transcript Count Threshold",
    y_axis_title="# of cells remaining in cluster"
)

In [ ]:

transcript_count_threshold = transcript_count_thresholds[0]

# Calculate the cell loss (% cells remaining) at each threshold to make a normalized trace
final_cluster_loss = cluster_size_from_most_to_least.iloc[:,-1]/cluster_size_from_most_to_least.iloc[:,0]
cluster_losses = cluster_size_from_most_to_least.values[:, 1:]/cluster_size_from_most_to_least.values[:, 0].reshape((-1, 1))
cluster_losses[numpy.isnan(cluster_losses)] = 0

# Cluster the traces into two - noise and not
noise_clusterer = AgglomerativeClustering(n_clusters=2)
noise_clusters = noise_clusterer.fit_predict(cluster_losses)

# Find the cluster that maintains the largest number of cells - this is our signal
signal_cluster = noise_clusters[numpy.where(final_cluster_loss.values == final_cluster_loss.values.max())[0][0]]

# Initialize a boolean array with all false
valid_cells = numpy.array([False]*filtered_cell_gene_counts.num_rows)

print("Filtering out cluster(s): %s" % str(numpy.argwhere(noise_clusters != signal_cluster)[:, 0]))
print("Keeping cluster(s): %s" % str(numpy.argwhere(noise_clusters == signal_cluster)[:, 0]))

# Loop through all clusters and mark any cells that are part of a valid cluster as valid
for cluster in range(num_clusters):
    
    if noise_clusters[cluster] == signal_cluster:
        valid_cells = valid_cells | (clusters == cluster)
        
cluster_filtered_cell_gene_counts = filtered_cell_gene_counts[valid_cells, :]

print("Filtered out %i cells out of %i" % (len(valid_cells) - valid_cells.sum(), len(valid_cells)))